In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
from IPython.display import display

In [4]:
# Load Dataset 
df = pd.read_csv('../data/df_file.csv')

In [5]:
# Display Basic Info
print("Dataset Overview")
display(df.head())

Dataset Overview


,Text,Label
0,Budget to set scene for election\n \n Gordon B...,0
1,Army chiefs in regiments decision\n \n Militar...,0
2,Howard denies split over ID cards\n \n Michael...,0
3,Observers to monitor UK election\n \n Minister...,0
4,Kilroy names election seat target\n \n Ex-chat...,0


In [6]:
# Display Dataset Shape
print("\nDataset Shape:")
print(f"Number of Rows: {df.shape[0]}")
print(f"Number of Columns: {df.shape[1]}")


Dataset Shape:
Number of Rows: 2225
Number of Columns: 2


In [7]:
# Display Column Names and Data Types
print("\nColumn Names and Data Types:")
print(df.dtypes)


Column Names and Data Types:
Text     object
Label     int64
dtype: object
